In [2]:
import pandas as pd
from setuptools.dist import sequence

books=pd.read_csv('books_cleaned.csv')

In [3]:
books['categories'].value_counts().reset_index()

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
...,...,...
474,Conspiracies,1
475,Brothers and sisters,1
476,Rock musicians,1
477,Community life,1


In [4]:
books['categories'].value_counts().reset_index().query("count > 50")

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Philosophy,117
6,Religion,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [5]:
category_mapping={'Fiction':'Fiction',
                  'juvenile fiction':"Children's Fiction",
                  'Biography & Autobiography':'Nonfiction',
                  'Science Fiction':'Nonfiction',
                  'Mystery & Suspense':'Nonfiction',
                  'Horror & Fantasy':'Nonfiction',
                  'Romance':'Nonfiction',
                  'Thriller & Suspense':'Nonfiction',
                  'Historical Fiction':'Nonfiction',
                  'Western':'Nonfiction',
                  'Science & Technology':'Nonfiction',
                  'History':'Nonfiction',
                  'poetry':'Nonfiction',
                  'Drama':'Fiction',
                  'Comics & Graphic Novels':'Fiction',
                  'juvenile Nonfiction':"Children's Nonfiction",}

books['simple_categories']=books['categories'].map(category_mapping)

In [6]:
books['simple_categories'].value_counts()

simple_categories
Fiction       2313
Nonfiction     518
Name: count, dtype: int64

In [7]:
from transformers import pipeline

fiction_categories=["Fiction","Nonfiction"]

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

C:\Users\Admin\PycharmProjects\LLM(Book-recomendor)\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [10]:
sequence =  books.loc[books["simple_categories"]=="Fiction","description"].reset_index(drop=True)[0]

In [9]:
pipe(sequence, fiction_categories)

NameError: name 'pipe' is not defined

In [ ]:
import numpy as np

max_index=np.argmax(pipe(sequence, fiction_categories)['scores'])
max_label=pipe(sequence, fiction_categories)['labels'][max_index]
max_label

In [ ]:
def generate_predictions(sequence,categories):
    predictions=pipe(sequence, categories)["labels"]
    scores=pipe(sequence, categories)["scores"]
    return predictions, scores

In [ ]:
from tqdm import tqdm

actual_cats=[]
predicted_cats=[]

for i in tqdm(range(0,300)):
    sequence=books.loc[books["simple_categories"]=="Fiction","description"].reset_index(drop=True)[i]
    predicted_cats += generate_predictions(sequence,fiction_categories)
    actual_cats += ["Fiction"]

In [ ]:
for i in tqdm(range(0,300)):
    sequence=books.loc[books["simple_categories"]=="Nonfiction","description"].reset_index(drop=True)[i]
    predicted_cats += generate_predictions(sequence,fiction_categories)
    actual_cats += ["Nonfiction"]

In [ ]:
predictions_df= pd.DataFrame({"actual_categories":actual_cats,"predicted_categories":predicted_cats})

In [ ]:
predictions_df

In [ ]:
predictions_df["correct_predictions"]= (
    np.where(predictions_df["actual_categories"]==predictions_df["predicted_categories"],1,0)

)

In [ ]:
predictions_df["correct_predictions"].sum()/len(predictions_df)

In [ ]:
isbns=[]
prediction_cats=[]

missing_cats=books.loc[books["simple_categories"].isna(),["isbn13", "description"]].reset_index(drop=True)

In [ ]:
for i in tqdm(range(0,len(missing_cats))):
    sequence=missing_cats["description"][i]
    predicted_cats += generate_predictions(sequence,fiction_categories)
    isbns += [missing_cats["isbn13"][i]]

In [ ]:
missing_predicted_df=pd.DataFrame({"isbn13":isbns,"predicted_categories":predicted_cats})

In [ ]:
missing_predicted_df

In [ ]:
books=pd.merge(books,missing_predicted_df,on="isbn13",how="left")
books["simple_categories"]= np.where(books["simple_categories"].isna(),books["predicted_categories"],books["simple_categories"])
books=books.drop(columns=["predicted_categories"])

In [ ]:
books.to.csv("books_with_categories.csv", index="False")